# Początkowa szeroka biblioteka modeli

Biblioteka modeli została skonstruowana w oparciu o hierarchię skuteczności, stawiając zdecydowany nacisk na Gradient Boosting (XGBoost, CatBoost) oraz Lasy Losowe. Wybór ten podyktowany jest ich wysoką zdolnością generalizacji na surowych danych. Jednocześnie, w celu zapewnienia różnorodności predykcyjnej i stabilności ensamblu, pulę uzupełniono o modele o odmiennych biasach indukcyjnych (SVM, Regresja Logistyczna, Naive Bayes).

Konfiguracja hiperparametrów nie opiera się na metodzie Brute Force, lecz na świadomym projektowaniu wariantów (Model Design Strategy):

Balans Złożoności: Parametry mogące prowadzić do overfittingu (np. max_depth=10 w boostingach) są zawsze kontrowane parametrami regularyzacyjnymi (np. gamma, reg_lambda, min_samples_leaf), co zapewnia stabilność uczenia.

Adaptacyjność: Zbiór konfiguracji zawiera zarówno "płytkie" estymatory dla prostych zadań, jak i rozbudowane struktury dla danych wielowymiarowych.

Obsługa Nierównowagi: We wszystkich dostępnych algorytmach systemowo uwzględniono wagi klas (parametry typu class_weight lub strategie samplingu), minimalizując wpływ dysproporcji w danych treningowych na wynik końcowy.

Informacje o paramerach i ich cechach pochodzą bezpośrednio z dokumentacji każdego z modeli.

### Modele liniowe

**Parametry - opis:**
* **penalty** - określa rodzaj regularyzacji, czyli karę za zbyt duże współczynniki modelu.
* **C** - odwrotność siły regularyzacji, określa jak bardzo pozwalamy modelowi ufać danym.
* **solver** - algorytm optymalizacji, który znajduje najlepsze współczynniki.
* **class_weight** - określa jak ważne są poszczególne klasy podczas uczenia modelu.
* **max_iter** - maksymalna liczba kroków które solver może wykonać podczas uczenia, dobierany odpowiednio do pozostałych parametrów modelu.

**Parametry ogólne:**
* **penalty**: l1, **l2 (default)**, elasticnet, none
* **C**: **1.0 (default)**
* **solver**: saga, **lbfgs (default)**, liblinear, svd, lsqr
* **class_weight**: balanced, **None (default)**
* **max_iter**: **100 (default)**

**Opis zastosowania:**

---
* **l1** -> zeruje nieistotne cechy; przydatne przy dużych, zaszumionych zbiorach, gdy podejrzewamy że tylko częśc cech jest istotna.
* **l2** -> karze duże wagi, ale nie zeruje ich; wybieramy, gdy nie wiemy wiele o danych, gdy cechy wydaja sie w miarę sensowen, jest to stabliny wariant bazowy.
* **l1 + l2 (elasticnet)** -> zeruje cechy, ale grupowo; dla danych skorelowanych, gdy wiele cech jest podobnych (np. cechy one-hot), jak chcemy uniknąć losowego wyboru jednej z wielu podobnych cech.
---
* **małe C (np. 0.01, 0.1)** -> silna regularyzacja; dobre gdy mało danych, dużo cech, dużo szumu, dane wysoko wymiarowe, ryzyko underfittingu.
* **średnie C (np. 0.5, 1)** -> stabilne rozwiązanie; gdy nie wiemy nic o jakości cech.
* **duże C (np. 10, 100)** -> słaba regularyzacja, model mocno dopasowuje się do danych; dobre dla bardzo dużych danych, niskiego poziomu szumu, ryzyko overfittingu.
---
* **saga** -> aktualizuje wagi na małych porcjach danych, obsługuje l1 i elasticnet; dobry dla dużych zbiorów danych, wysokiej liczby cech.
* **lbfgs** -> algorytm quasi-newtonowski, klasyczny, stabilny wariant, obsługuje tylko l2; dobry jako domyślny solver gdy nie wiemy nic o danych.
* **liblinear** -> oparty o optymalizację liniową; dobry dla małych zbiorów.
* **LDA** -> solvery **lsqr** oraz **svd** są zalecane, gdy mamy więcej cech niż próbek.
* **lsqr** -> gdy dane są skorelowane, **svd**: dla wysokowymiarowych, dużo cech.
---
* **balanced** -> próbki klasy mniejszościowej dostają większą wagę; dobre gdy nie znamy rozkładu klas, gdy mozliwa duża nierównowaga.

---

**Wybrane kombinacje (LogisticRegression):**

1. **Standardowy model z solverem SAGA** (dobre dla dużych danych):
   "penalty": "l2", "C": 1.0, "solver": "saga", "class_weight": "balanced", "max_iter": 2000

2. **Klasyczny solver LBFGS** (stabilny):
   "penalty": "l2", "C": 1.0, "solver": "lbfgs", "class_weight": "balanced", "max_iter": 2000

3. **Regularyzacja L1** (selekcja cech):
   "penalty": "l1", "C": 0.5, "solver": "saga", "class_weight": "balanced", "max_iter": 2000

4. **ElasticNet** (dane skorelowane):
   "penalty": "elasticnet", "l1_ratio": 0.5, "C": 0.1, "solver": "saga", "class_weight": "balanced", "max_iter": 3000

5. **Silna regularyzacja dla mniejszych zbiorów** (małe, trudne dane):
   "penalty": "l2", "C": 0.01, "solver": "liblinear", "class_weight": "balanced", "max_iter": 1000

**LinearDiscriminantAnalysis (LDA):**
* "solver": "svd" (dużo cech)
* "solver": "lsqr", "shrinkage": "auto" (dane skorelowane)

**RidgeClassifier:**
* "alpha": 1.0, "class_weight": "balanced", "solver": "auto"

---

### Drzewa decyzyjne

**Parametry - opis:**
* **max_depth** - maksymalna głębokość drzewa, określa złożonośc modelu.
* **min_samples_leaf** - minimalna liczba próbek w liściu, dobry parametr do walki z overfittingiem.
* **min_samples_split** - minimalna liczba próbek do podziału, zabobiega dzieleniu bardzo małych grup.
* **criterion** - określa miarę jakości podziału węzłów drzewa decyzyjnego (jak mierzymy dobry split).
* **class_weight** - wagi klas.


**Parametry bazowe:**
* **max_depth**=None, **min_samples_leaf**=1, **min_samples_split**=2
* **criterion**='gini'
* **class_weight**=None


**Opis zastosowania:**

---
* **max_depth (1-5)** -> proste relacje, dla małych danych i małego szumu.
* **max_depth (15-20)** -> bardzo złożone relacje, gdy dużo danych.
* **max_depth (None)** -> brak limitu, typowo do eksploracji.
---
* **min_samples_leaf(1)** -> maksymalna ekspresja, gdy dużo danych.
* **min_samples_leaf(>10)**  -> mało danych, gdy jest szum.
---
* **gini** -> mierzy jak często losowo wybrana próbka byłaby źle sklasyfikowana, gdybyśmy losowo zgadywali klasę zgodnie z rozkładem w węźle; dobre dla dużych danych, gdy zależy na szybkości.

---


**Konfiguracje:**

1. **Bardzo płytkie (Decision Stump):**
   "max_depth": 1, "criterion": "gini" -> jedna reguła decyzyjna; dla trywialnych danych.

2. **Bezpieczny wybór (Średnia złożoność):**
   "max_depth": 10, "min_samples_leaf": 5, "class_weight": "balanced"

3. **Głębokie drzewo (Wysoka złożoność):**
   "max_depth": 20, "min_samples_split": 10, "class_weight": "balanced", "criterion": "gini" -> potencjał dla złożonych granic, ryzyko overfittingu.

4. **Bez ograniczeń (Eksploracja):**
   "max_depth": null, "min_samples_leaf": 1, "class_weight": null -> ryzyko przeuczenia, do weryfikacji.

---

### Drzewa losowe (Random Forest)

**Parametry bazowe:**
* **n_estimators**=100, **criterion**='gini', **max_depth**=None
* **min_samples_split**=2, **min_samples_leaf**=1

**Konfiguracje:**

1. **Szybki, prosty, płytki:**
   "n_estimators": 100, "max_depth": 10, "n_jobs": 1, "class_weight": "balanced", "random_state": 42

2. **Pełne drzewo (Mocny model):**
   "n_estimators": 300, "max_depth": null, "min_samples_leaf": 2 -> brak ryzyka overfittingu (asymptotycznie).

3. **Głębokie z ograniczeniami (Bezpieczniejsze):**
   "n_estimators": 500, "max_depth": 15, "min_samples_leaf": 5, "class_weight": "balanced" -> większe min_samples_leaf patrzy bardziej na ogół.

4. **Eksploracja entropii:**
   "n_estimators": 200, "criterion": "entropy", "max_depth": 25, "min_samples_split": 5 -> inny rodzaj kryterium dla różnorodności.

**ExtraTreesClassifier:**
1. **Wariant zrównoważony:** "n_estimators": 200, "max_depth": 12, "min_samples_leaf": 2, "bootstrap": true, "class_weight": "balanced"
2. **Wariant głęboki:** "n_estimators": 400, "max_depth": null, "criterion": "entropy"

---

### XGBoost

**Parametry - opis:**
* **learning_rate** - krok uczenia, określa jak bardzo każde kolejne drzewo wpływa na wynik.
* **n_estimators** - liczba drzew w esemble.
* **max_depth** - maksymalna głębokość pojedyńczego drzewa.
* **subsample** - procent próbek używanych do budowy każdego drzewa.
* **colsample_bytree** - procent cech używanych do budowy drzewa.
* **reg_lambda** - regularyzacja wag liści L2, zmniejsza wagi liści.
* **reg_alpha** - regularyzacja wag liści L1, zeruje nieistotne liście.
* **gamma** - próg opłacalności podziału w drzewie, dobry do walki z overfittingiem.
* **min_child_weight** - minimalna suma wag próbek w liściu, chroni przed liścmi opartymi o pojedyńcze próbki.

**Parametry (defaults):**
* **learning_rate** (0.3), **n_estimators** (rounds), **max_depth** (6)
* **subsample** (1), **colsample_bytree** (1)
* **reg_lambda** (1), **reg_alpha** (0)
* **gamma** (0)
* **min_child_weight** (1)

**Opis zastosowania:**

---
* **learning_rate (0.01-0.05)** -> bardzo wolne, stabilne uczenie; dobre dla dużych i czystych danych.
* **learning_rate (0.1-0.3)** -> szybsze i bardzoiej agresywne; lepsze dla małych zbiorów.
---
* **subsample (0.7-0.9)** -> umiarkowana regularyzacja, gdy pojawia się overfitting, dobrze jest zmniejszać; dla wiekszych danych mniejsze.
* **colsample_bytree (0.6-0.8)** -> standard, kluczowy przy danych wysokowymairowych.
* **colsample_bytree (0.4-0.6)** -> silna redukcja korelacji; gdy jest bardzo dużo cech.
---
* **gamma (0)** -> każdy sensowny split; dla dużych danych.
* **gamma (0.1-.05)** -> odzruca słabe splity.

---

**Konfiguracje:**

1. **Płytki, szybko uczący się:**
   "n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "subsample": 0.8

2. **Głębszy, umiarkowana szybkość:**
   "n_estimators": 400, "learning_rate": 0.05, "max_depth": 7, "subsample": 0.9, "colsample_bytree": 0.8

3. **Średni model (Baseline):**
   "n_estimators": 150, "max_depth": 6, "learning_rate": 0.1

4. **Slow Learner (Bardzo wolna nauka):**
   "n_estimators": 1000, "learning_rate": 0.01, "max_depth": 6, "subsample": 0.7, "colsample_bytree": 0.7

5. **Głęboki z ograniczeniami:**
   "n_estimators": 200, "learning_rate": 0.05, "max_depth": 10, "min_child_weight": 5, "gamma": 0.2

6. **Regularyzacja (L1/L2):**
   "n_estimators": 300, "learning_rate": 0.05, "max_depth": 5, "reg_alpha": 0.5, "reg_lambda": 1.0

---

### CatBoostClassifier

**Parametry - opis:**
* **iterations** - liczba drzew dodanych do ensemble (odpowiednik n_estimators), więcej iteracji = większa zdolność dopasowania, ale wolniejsze uczenie.
* **depth** * - głębokość symetrycznego dzrewa.
* **l2_leaf_reg** - kara za duże wagi, zapobiega przeuczeniu.
* **bagging_temperature** - wagi obserwacji z rozkładu wykładniczego (Bayesian Bootstrap), kontroluje jak losowo wybierane są próbki, 0 -> brak losowości.
* **random_strength** - losowość przy wyborze podziału.

**Parametry (defaults):**
* **iterations** (1000), **depth** (6), **l2_leaf_reg** (3)
* **bagging_temperature** (1), **random_strength** (1), **learning_rate** (0.3)

**Opis zastosowania:**

---
* **bagging_temperature (0.5-1)** -> umairkowana losowość, im większa wartośc tym większa losowość; większa dobra dla dużych danych.
---

**Konfiguracje:**

1. **Standardowy:**
   "iterations": 150, "depth": 6, "learning_rate": 0.15

2. **Wolniejszy z regularyzacją:**
   "iterations": 500, "depth": 8, "learning_rate": 0.05, "l2_leaf_reg": 3

3. **Mocny model (Intensive):**
   "iterations": 1000, "learning_rate": 0.03, "depth": 8, "l2_leaf_reg": 5, "bagging_temperature": 1

4. **Długodystansowiec:**
   "iterations": 2000, "learning_rate": 0.02, "depth": 8, "l2_leaf_reg": 5, "bagging_temperature": 1

5. **Łagodniejszy wariant:**
   "iterations": 800, "learning_rate": 0.04, "depth": 8, "l2_leaf_reg": 5, "bagging_temperature": 1

6. **Płytki (Small):**
   "iterations": 600, "depth": 4, "learning_rate": 0.1, "l2_leaf_reg": 3

7. **Deep Slow (Eksperymentalny):**
   "iterations": 1500, "depth": 10, "learning_rate": 0.01, "l2_leaf_reg": 9, "random_strength": 1

---

### KNN (K-Nearest Neighbors)

**Parametry - opis:**
* **n_neighbours** - liczba najbliższych obserwacji ze zbioru treningowego, które są brane pod uwagę przy wyznaczaniu klasy.
* **weights** - określa jak liczy się głosy sąsiadów.
* **p** -definiuje jak liczona jest odległość.

**Parametry (defaults):**
* **n_neighbours** (5), **weights** ("uniform"), "distance" 
* **p** (2)

**Opis zastosowania:**

---
* **uniform** -> każdy sąsiad ma taki sam wpływ, niezależnie od odległości (uśrednia).
* **distance** -> głos jest ważony odwrotnie proporcjonalnie do odległości (słucha najbliższych); lepsze przy nierównej gęstości danych, odporne na szum.
---
* **p=1 (Manhattan)** -> odporne na wartości odstające, większa odporność na szum.
* **p=2** -> standard dla danych ciągłych, wrażliwa na skalę.


**Konfiguracje:**
1. **Średnie K, wagi odległościowe:** "n_neighbors": 15, "weights": "distance"
2. **Duże K:** "n_neighbors": 25, "weights": "distance"
3. **Domyślny:** "n_neighbors": 5, "weights": "uniform"
4. **Wysokowymiarowy:** "n_neighbors": 50, "weights": "distance", "p": 1 

---

### SVC (Support Vector Classifier)

**Parametry - opis:**
* **C** - parametr regularizacji, kontroluje karę za błędną klasyfikację próbek treningowych.
* **kernel** - określa postać transformacji przestrzeni cech, w której budowana jest hiperpłaszczyzna separująca.
* **gamma** - określa zasięg wpływu pojedynczej próbki treningowej w przestrzeni jądra.
* **probability** - Określa, czy model ma zwracać prawdopodobieństwa klas zamiast samych etykiet.
* **class_weight** - wagi przypisane klasom, używane do modyfikacji funkcji straty w przypadku niezbalansowanych danych.
* **cache_size** - rozmiar pamięci podręcznej (w MB) przeznaczonej na przechowywanie obliczeń macierzy jądra w trakcie uczenia.

**Parametry (defaults):**
* **C** (default 1.0), **kernel** (rbf), linear, poly
* **gamma** (scale), **probability** (False), True
* **class_weight** (None), "balanced", **cache_size** (200)

**Opis zastosowania:**

---
* **rbf** -> mapuje dane do nieskończenie wymiarowej przestrzeni, umożliwiając nieliniową separację; do danych nieliniowych, gdy nie znamy struktury danych
* **linear** -> buduje liniową hiperpłaszczyznę; dla danytch liniowo separowalnych, dużo cech, mało próbek, gdy dane prawie liniowe
* **poly** -> jądro wielomianowe; do niskowymiarowych danych
---
* **probabilty (True)** -> wymagane do AUC.
---


**Konfiguracje:**
1. **Bazowy o średnich parametrach**
    "max_depth": 7, "class_weight": "balanced","criterion": "entropy"

1. **Standardowy RBF:**
   "C": 1.0, "kernel": "rbf", "gamma": "scale", "probability": true, "class_weight": "balanced", "cache_size": 1000

2. **Liniowy (Soft Margin):**
   "C": 0.1, "kernel": "linear", "probability": true, "class_weight": "balanced"

3. **Wielomianowy:**
   "C": 1.0, "kernel": "poly", "degree": 3, "probability": true, "class_weight": "balanced"

4. **Hard Margin (Rygorystyczny):**
   "C": 10.0, "kernel": "rbf", "gamma": "scale", "probability": true

---

### Inne modele

* **sklearn.naive_bayes.GaussianNB**
* **sklearn.naive_bayes.BernoulliNB**: {"alpha": 0.5} oraz {"alpha": 1.0} (test wpływu regularyzacji na dane binarne)
* **sklearn.ensemble.AdaBoostClassifier**: {"n_estimators": 100, "learning_rate": 1.0} oraz {"n_estimators": 200, "learning_rate": 0.1} (szybkie i agresywne uczenie vs mniejsze kroki i stabilniej)
* **sklearn.ensemble.GradientBoostingClassifier**: {"n_estimators": 200, "learning_rate": 0.1, "max_depth": 3, "subsample": 0.8, "validation_fraction": 0.1, "n_iter_no_change": 10} (klasyczne, stabline)
* **sklearn.ensemble.HistGradientBoostingClassifier**: {"max_iter": 200, "learning_rate": 0.1, "max_depth": 10, "l2_regularization": 0.5} (szybki, skalowany na wieksze dane)

W celu zapewnienia pełnej powtarzalności eksperymentu, parametry random_state zastosowano we wszystkich algorytmach niedeterministycznych. Modele takie jak LDA, KNN oraz naiwne klasyfikatory Bayesa (GaussianNB, BernoulliNB) działają w sposób deterministyczny, dlatego nie wymagają tego ustawienia.

W przypadku Lasów Losowych zdecydowano się na pracę jednowątkową (n_jobs=1), co eliminuje ryzyko potencjalnych różnic numerycznych wynikających z wielowątkowości, kosztem dłuższego czasu trenowania.

Dla algorytmu CatBoost skonfigurowano parametry techniczne: wyłączono zapis plików na dysku (allow_writing_files=False) oraz wyciszono standardowe wyjście modelu (verbose=0).

Powyższe konfiguracja zostały zapisane w pliku models_for_test.json do dalszej selekcji.